# Term 7 project: Inventory management
### Link to Original Github Repo: https://github.com/eg4000/SKU110K_CVPR19
### SKU-110K dataset can be found here: https://www.kaggle.com/datasets/thedatasith/sku110k-annotations/data 

In [1]:
import os
import tensorflow as tf
print(tf.__version__)
print("GPU Available:", tf.config.experimental.list_physical_devices('GPU'))
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
import dotenv

# create your own .env file and add in your directory in the format
# SKU_DATASET_DIR = r"your_directory"
dotenv.load_dotenv()
sku_dataset_dir =  os.getenv("SKU_DATASET_DIR")
annotations_dir = Path(sku_dataset_dir) / 'annotations'
images_path = Path(sku_dataset_dir) / 'images'

2.10.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Delete corrupted images
- List: https://github.com/eg4000/SKU110K_CVPR19/issues/99#issuecomment-988886374

In [2]:
CORRUPTED_IMAGES = {
    "train": (
              "train_1239.jpg",
              "train_2376.jpg",
              "train_2903.jpg",
              "train_2986.jpg",
              "train_305.jpg",
              "train_3240.jpg",
              "train_340.jpg",
              "train_3556.jpg",
              "train_3560.jpg",
              "train_3832.jpg",
              "train_38.jpg",
              "train_4222.jpg",
              "train_5007.jpg",
              "train_5137.jpg",
              "train_5143.jpg",
              "train_5762.jpg",
              "train_5822.jpg",
              "train_6052.jpg",
              "train_6090.jpg",
              "train_6138.jpg",
              "train_6409.jpg",
              "train_6722.jpg",
              "train_6788.jpg",
              "train_737.jpg",
              "train_7576.jpg",
              "train_7622.jpg",
              "train_775.jpg",
              "train_7883.jpg",
              "train_789.jpg",
              "train_8020.jpg",
              "train_8146.jpg",
              "train_882.jpg",
              "train_903.jpg",
              "train_924.jpg"
             ),
    "validation": (
              "val_147.jpg",
              "val_286.jpg",
              "val_296.jpg",
              "val_386.jpg"
             ),
    "test": (
              "test_132.jpg",
              "test_1346.jpg",
              "test_184.jpg",
              "test_1929.jpg",
              "test_2028.jpg",
              "test_22.jpg",
              "test_2321.jpg",
              "test_232.jpg",
              "test_2613.jpg",
              "test_2643.jpg",
              "test_274.jpg",
              "test_2878.jpg",
              "test_521.jpg",
              "test_853.jpg",
              "test_910.jpg",
              "test_923.jpg"
             ),             
}

for subset, filenames in CORRUPTED_IMAGES.items():
    for filename in filenames:
        file_path = os.path.join(images_path, filename)
        if os.path.exists(file_path):
            os.remove(file_path)
            #print(f"Removed: {file_path}")
        else:
            pass
            #print(f"File not found: {file_path}")


## Preprocessing
- addressing OOM issue: reduce image size(4160x2336 to 416, 233), limit batch size, use tensorflow api

In [3]:
from PIL import Image
import io

def load_and_preprocess_data(annotations_file, images_path, subset_size=0.01, img_size=(416, 233), batch_size=256):
    df = pd.read_csv(annotations_file)
    subset = "train" if "train" in str(annotations_file) else "validation" if "val" in str(annotations_file) else "test"
    corrupted_images = CORRUPTED_IMAGES.get(subset, [])
    df = df[~df.iloc[:, 0].isin(corrupted_images)]
    df = df.sample(frac=subset_size, random_state=42).reset_index(drop=True)

    images_path = str(images_path)  # Convert images_path to string

    def preprocess(image_name, x1, y1, x2, y2, w, h):
        image_name = image_name.numpy().decode('utf-8')
        image_path = os.path.join(images_path, image_name)

        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, img_size)  # Resize image
        img = tf.image.convert_image_dtype(img, tf.float32)  # Normalize to [0, 1]

        x1, x2 = x1 / w, x2 / w
        y1, y2 = y1 / h, y2 / h

        bbox = tf.convert_to_tensor([y1, x1, y2, x2], dtype=tf.float32)  # TensorFlow format: [ymin, xmin, ymax, xmax]
        return img, bbox

    # Convert dataframe columns to tensors
    image_names = tf.convert_to_tensor(df[df.columns[0]].values, dtype=tf.string)
    x1 = tf.convert_to_tensor(df[df.columns[1]].values.astype(float), dtype=tf.float32)
    y1 = tf.convert_to_tensor(df[df.columns[2]].values.astype(float), dtype=tf.float32)
    x2 = tf.convert_to_tensor(df[df.columns[3]].values.astype(float), dtype=tf.float32)
    y2 = tf.convert_to_tensor(df[df.columns[4]].values.astype(float), dtype=tf.float32)
    w = tf.convert_to_tensor(df[df.columns[6]].values.astype(float), dtype=tf.float32)
    h = tf.convert_to_tensor(df[df.columns[7]].values.astype(float), dtype=tf.float32)

    dataset = tf.data.Dataset.from_tensor_slices((image_names, x1, y1, x2, y2, w, h))
    dataset = dataset.map(lambda img, x1, y1, x2, y2, w, h: tf.py_function(preprocess, [img, x1, y1, x2, y2, w, h], [tf.float32, tf.float32]))
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

## Load data

In [4]:
subset_size = 0.1  # You can adjust this value as needed
img_size = (416, 233)  # Resize images to 224x224
batch_size = 64  # Number of samples per batch

train_data = load_and_preprocess_data(annotations_dir / 'annotations_train.csv', images_path, subset_size, img_size, batch_size)
val_data = load_and_preprocess_data(annotations_dir / 'annotations_val.csv', images_path, subset_size, img_size, batch_size)
test_data = load_and_preprocess_data(annotations_dir / 'annotations_test.csv', images_path, subset_size, img_size, batch_size)

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-11-21 02:31:13.743859: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-21 02:31:13.744307: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Visualise data

In [5]:
data_num, set = 0, 'test' 
image_num = sku_dataset_dir + rf'\images\{set}_{data_num}.jpg'
annotations_dir = Path(sku_dataset_dir) / 'annotations'
annotation_set_df = pd.read_csv(sku_dataset_dir + rf'\annotations\annotations_{set}.csv')
annotation_num = annotation_set_df[annotation_set_df.iloc[:, 0] == f'{set}_{data_num}.jpg']

def display_image_with_annotations(image_path, annotations_df):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(figsize=(16, 12))
    ax.imshow(img)

    # Add the bounding boxes
    for index, row in annotations_df.iterrows():
        x1, y1, x2, y2 = row[1], row[2], row[3], row[4]
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.axis('off')
    plt.show()

# Display the image with annotations
display_image_with_annotations(image_num, annotation_num)

NameError: name 'SKU_DATASET_DIR' is not defined

## Train

In [7]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers.experimental import preprocessing

EPOCHS = 5

# VGG16 Implementation

In [93]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the VGG16 layers

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1
)

model = models.Sequential(
    [
        Input(shape=(416, 233, 3)),
        preprocessing.Resizing(224, 224, interpolation="bilinear"),
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(
            4, activation="sigmoid"
        ),  # 4 values for bounding box (x, y, width, height)
    ]
)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ],
)

history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=val_data,
    callbacks=[cp_callback]
)
model.save("my_model_VGG.keras")

2023-11-20 22:54:15.078033: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - ETA: 0s - loss: 7.1306 - auc_24: 0.7101 - precision_22: 1.0000 - recall_22: 0.4093

2023-11-20 22:55:41.621072: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: saving model to training_1/cp.ckpt
19/19 [==============================] - 99s 5s/step - loss: 7.1306 - auc_24: 0.7101 - precision_22: 1.0000 - recall_22: 0.4093 - val_loss: 7.3528 - val_auc_24: 0.7039 - val_precision_22: 1.0000 - val_recall_22: 0.3994


In [94]:
from tensorflow.keras.models import load_model

model = load_model('my_model_VGG.keras')
iou = tf.keras.metrics.MeanIoU(num_classes=2) 

eval_results = model.evaluate(val_data)
print("Loss on validation data:", eval_results)

predictions = model.predict(val_data)

for images, true_boxes in val_data:
    # Get model predictions
    pred_boxes = model.predict(images)
    # Update IoU metric
    iou.update_state(true_boxes, pred_boxes)

# Get the final IoU
average_iou = iou.result().numpy()
print(f'Average IoU: {average_iou}')

2023-11-20 22:57:48.939331: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 7s 2s/step - loss: 7.3528 - auc_24: 0.7039 - precision_22: 1.0000 - recall_22: 0.3994
Loss on validation data: [7.352823734283447, 0.7039105892181396, 1.0, 0.39944133162498474]


2023-11-20 22:57:55.494481: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 5s 2s/step


2023-11-20 22:58:04.106330: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 275ms/step


2023-11-20 22:58:06.039677: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 293ms/step
Average IoU: 0.3156541883945465


# ResNet50 Implementation

In [8]:
base_model_2 = ResNet50(
    weights="imagenet",
    include_top=False,  # use our own input layer for transfer learning
    input_shape=(224, 224, 3),
)

base_model_2.trainable = False

checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1
)

resnet_model = models.Sequential(
    [
        Input(shape=(416, 233, 3)),
        preprocessing.Resizing(224, 224, interpolation="bilinear"),
        base_model_2,
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(
            4, activation="sigmoid"
        ),  # 4 values for bounding box (x, y, width, height)
    ]
)

resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
    ],
)

history = resnet_model.fit(
    train_data, epochs=EPOCHS, validation_data=val_data, callbacks=[cp_callback]
)
model.save("my_model_RESNET.keras")

Epoch 1/5


2023-11-21 02:32:21.769711: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1881/1881 [==============================] - ETA: 0s - loss: 7.3793 - auc_1: 0.7801 - precision_1: 0.9992 - recall_1: 0.7068

2023-11-21 04:19:03.197133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: saving model to training_2/cp.ckpt
1881/1881 [==============================] - 6875s 4s/step - loss: 7.3793 - auc_1: 0.7801 - precision_1: 0.9992 - recall_1: 0.7068 - val_loss: 7.4381 - val_auc_1: 0.7594 - val_precision_1: 0.9989 - val_recall_1: 0.7513
Epoch 2/5
1881/1881 [==============================] - ETA: 0s - loss: 7.3994 - auc_1: 0.7858 - precision_1: 0.9992 - recall_1: 0.7237
Epoch 2: saving model to training_2/cp.ckpt
1881/1881 [==============================] - 6909s 4s/step - loss: 7.3994 - auc_1: 0.7858 - precision_1: 0.9992 - recall_1: 0.7237 - val_loss: 7.4395 - val_auc_1: 0.7596 - val_precision_1: 0.9989 - val_recall_1: 0.7519
Epoch 3/5
1881/1881 [==============================] - ETA: 0s - loss: 7.4164 - auc_1: 0.7895 - precision_1: 0.9992 - recall_1: 0.7513
Epoch 3: saving model to training_2/cp.ckpt
1881/1881 [==============================] - 6967s 4s/step - loss: 7.4164 - auc_1: 0.7895 - precision_1: 0.9992 - recall_1: 0.7513 - val_loss: 7.4395 - val_auc

KeyboardInterrupt: 

In [99]:
eval_results = model.evaluate(val_data)
print("Loss on validation data:", eval_results)

from tensorflow.keras.models import load_model

model = load_model('my_model_RESNET.keras')
iou = tf.keras.metrics.MeanIoU(num_classes=2) 

predictions = model.predict(val_data)

for images, true_boxes in val_data:
    # Get model predictions
    pred_boxes = model.predict(images)
    # Update IoU metric
    iou.update_state(true_boxes, pred_boxes)

# Get the final IoU
average_iou = iou.result().numpy()
print(f'Average IoU: {average_iou}')

2/2 [==============================] - 6s 2s/step - loss: 7.3528 - auc_24: 0.7039 - precision_22: 1.0000 - recall_22: 0.3994
Loss on validation data: [7.352823734283447, 0.7039105892181396, 1.0, 0.39944133162498474]


2023-11-20 23:00:22.141202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 6s 2s/step


2023-11-20 23:00:31.528740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 292ms/step


2023-11-20 23:00:33.506038: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 375ms/step
Average IoU: 0.3156541883945465
